# Pipeline

starting out with sklearn I want to create my own pipeline similar to
  - https://michelleful.github.io/code-blog/2015/06/20/pipelines/
  - http://zacstewart.com/2014/08/05/pipelines-of-featureunions-of-pipelines.html
  
 but face some problems. In this sample notebook I formulate 3 Questions. Hopefully you can help me out. 
 
 Thanks a lot.

In [1]:
import pylab as P
%pylab inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cm as cm
plt.style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


Load  the data.

In [2]:
mydf = pd.DataFrame({'classLabel':[0,0,0,1,1,0,0,0],
                   'categorical':[7,8,9,5,7,5,6,4],
                   'numeric1':[7,8,9,5,7,5,6,4],
                   'numeric2':[7,8,9,5,7,5,6,"N.A"]})
mydf

,categorical,classLabel,numeric1,numeric2
0,7,0,7,7
1,8,0,8,8
2,9,0,9,9
3,5,1,5,5
4,7,1,7,7
5,5,0,5,5
6,6,0,6,6
7,4,0,4,N.A


However I only want to pass `numeric1` to my classifier.

#### Question1:

Do all transformers require X and y to already be separated? As I will be required to perform some cleanup for my real datast which might be easier for the whole dataframe.

transform to X, y representation as this is the standard for a pipeline

In [3]:
def transform(data): 
    # Extract the data
    X = data.drop(['classLabel'],  axis=1)
    
    # Extract the labels
    y = data['classLabel']
    
    return X, y

In [4]:
X, y = transform(mydf)

Imports for the real pipeline functions

In [5]:
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler, FunctionTransformer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn import cross_validation as cv
from sklearn.metrics import make_scorer, cohen_kappa_score

In [6]:
def extractSpecificColumn(X, columns):
    return X[columns]

In [7]:
columnsNumber = ['numeric1']
columnsFactor = ['categorical']
#columnsFactorOrdered =[]
#columnsDate =[]

In [8]:
# todo perform featgure extractions in parallel
pipeline = Pipeline([
    ('features', FeatureUnion([
        ('continuous', Pipeline([
            ('numeric', FunctionTransformer(columnsNumber)),
            ('scale', StandardScaler())
        ]))
                    #,
        #('factors', Pipeline([
        #    ('extract', ColumnExtractor(FACTOR_FIELDS)),
        #    ('one_hot', OneHotEncoder(n_values=5)),
        #    ('to_dense', DenseTransformer())
        #]))
    ], n_jobs=1)),
    ('estimator', RandomForestClassifier(n_estimators=50, criterion='entropy', n_jobs=-1))
])

#### todo warning set random seed at some point

In [9]:
kappaScore = make_scorer(cohen_kappa_score)

manual exploration

In [10]:
split = cv.StratifiedShuffleSplit(y, n_iter=1, test_size=0.2)

#### Question 2
how to get this to work?

In [11]:
for train_index, test_index in split:
    print(train_index)
    print(test_index)
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]
    print(X_test)

[7 4 2 3 5 0]
[1 6]


IndexError: indices are out-of-bounds

In [ ]:
# todo extract indices from stratified -> see above. should work similar to train_test_split

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
#model = pipeline.fit(data_train)

In [ ]:
#y_test = model.predict(data_test)

crossvalidation

In [12]:
 XoneColumn = X[columnsNumber]

#### Question 3
what is the problem here if I pass `XoneColumn`?

In [13]:
folds = cv.StratifiedShuffleSplit(y, test_size=0.2, n_iter=3)
cv.cross_val_score(pipeline, XoneColumn, y, cv=folds, scoring=kappaScore)

TypeError: 'list' object is not callable

improvement & tuning

no hard scoring, use probabilities and calibration https://jmetzen.github.io/2015-04-14/calibration.html

In [ ]:
#gridsearch CV